In [10]:
import BoW
import pandas as pd
import os.path
import datetime
from sklearn.model_selection import train_test_split

In [11]:
# Data Loading
path = os.getcwd()
parent_folder, current_folder = os.path.split(path)
df = pd.read_csv(parent_folder + '/0.Raw_data/train/Combined_News_DJIA_train.csv')   # please check if Training data is in the same location on your PC

In [3]:
# Data Cleaning and merging all headlines to one single column

subdf = df.iloc[:,2:27] 
subdf = subdf.applymap(str)
s = subdf.apply(lambda x: ' '.join(x), axis=1)

# replace the b' and b" which are in the beginning of some headlines
s = s.str.replace("b'","")
s = s.str.replace('b"','')

# create a new dataframe with all headlines and the overall word count
df1 = s.to_frame(name='headlines')
#df1['lengths'] = df1['headlines'].apply(len)
df1['date'] = pd.to_datetime(df['Date'])

In [4]:
df1.dtypes

headlines            object
date         datetime64[ns]
dtype: object

In [12]:
#X_train, X_val, y_train, y_val = train_test_split(df1, df['Label'], test_size=0.2, random_state=42)

In [19]:
val_size = 0.2
train_length = len(df1)*(1-val_size)
train = df1.loc[:train_length,:]
val = df1.loc[train_length:,:]

In [27]:
val.head()

,headlines,lengths,label,date
1268,An Indian spiritual leader who sparked controv...,3072,1.0,2013-08-22
1269,Orders To Destroy Guardian Hard Drives Came Di...,3054,1.0,2013-08-23
1270,Exclusive: CIA Files Prove America Helped Sadd...,3805,0.0,2013-08-26
1271,Hans Blix: Whether Obama in Syria or Bush in I...,2221,0.0,2013-08-27
1272,A campaign has been launched to encourage the ...,2392,1.0,2013-08-28


In [28]:
print(len(train))
print(len(val))

X_train = train.loc[:,train.columns != 'label']
X_val = val.loc[:,val.columns != 'label']
y_train = train['label']
y_val = val['label']

1268
316


In [21]:
df1.head()

,headlines,lengths,label,date
0,Georgia 'downs two Russian warplanes' as count...,2187,0.0,2008-08-08
1,Why wont America and Nato help us? If they won...,1566,1.0,2008-08-11
2,Remember that adorable 9-year-old who sang at ...,2111,0.0,2008-08-12
3,U.S. refuses Israel weapons to attack Iran: r...,1864,0.0,2008-08-13
4,All the experts admit that we should legalise ...,1785,1.0,2008-08-14


In [40]:
# Encoding the training data set
df_encode = BoW.bow_encoding(X_train['headlines'])
df_encode

<1268x10225 sparse matrix of type '<class 'numpy.int64'>'
	with 280329 stored elements in Compressed Sparse Row format>

In [48]:
def bow_encoding_val(s):
    '''
    Takes a pandas Series and transforms the strings to a Bag of Words matrix
    :param s: pandas series
    :return: Numpy Matrix
    '''
    import pickle
    with open('CountVectorizer.pk', 'rb') as f:
        BoW_WN = pickle.load(f)
    BoW_WN_matrix = BoW_WN.transform(s)

    return BoW_WN_matrix


def tfidf_encoding_val(s):
    '''
    Takes a series of Texts and transforms the strings to a TFIDF matrix
    :param s: pandas series
    :return: Numpy Matrix
    '''
    import pickle
    with open('TFIDFVectorizer.pk', 'rb') as f:
        tfidf_vectorizer_WN = pickle.load(f)
    tfidf_WN_matrix = tfidf_vectorizer_WN.transform(s)

    return tfidf_WN_matrix

In [38]:
# transforming
#from BoW import bow_encoding_val
df_encode_val = bow_encoding_val(X_val['headlines'])
df_encode_val

<316x10225 sparse matrix of type '<class 'numpy.int64'>'
	with 71748 stored elements in Compressed Sparse Row format>

In [43]:
df_encode_TFIDF = BoW.tfidf_encoding(X_train['headlines'])
df_encode_TFIDF

<1268x10225 sparse matrix of type '<class 'numpy.float64'>'
	with 280329 stored elements in Compressed Sparse Row format>

In [49]:
df_encode_TFIDF_val = tfidf_encoding_val(X_val['headlines'])
df_encode_TFIDF_val

<316x10225 sparse matrix of type '<class 'numpy.float64'>'
	with 71748 stored elements in Compressed Sparse Row format>

In [51]:
X_bow_train= pd.DataFrame(df_encode.toarray())
X_bow_val= pd.DataFrame(df_encode_val.toarray())
X_TFIDF_train= pd.DataFrame(df_encode_TFIDF.toarray())
X_TFIDF_val= pd.DataFrame(df_encode_TFIDF_val.toarray())

In [59]:
Y_train = train['label']
Y_val = val['label']

In [53]:
import os
os.getcwd()

'C:\\Users\\lundr\\DSRCompTwo\\2.Feature_engineering'

In [54]:
import os
os.chdir('..')

In [61]:
X_bow_train.to_csv('4.Data/X_bow_train_v1.csv')
X_bow_val.to_csv('4.Data/X_bow_val_v1.csv')
X_TFIDF_train.to_csv('4.Data/X_TFIDF_train_v1.csv')
X_TFIDF_val.to_csv('4.Data/X_TFIDF_val_v1.csv')



In [62]:
Y_train.to_csv('4.Data/Y_train_v1.csv')
Y_val.to_csv('4.Data/Y_val_v1.csv')

C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [60]:
len(Y_train)

1268